# Homework 9

# Groth16 Part 1

Use a QAP from an earlier homework as a starting point.

The trusted setup should generate the following values:

$$
\tau, \alpha, \beta
$$

which are the “toxic waste” (read this [https://www.coindesk.com/layer2/2022/01/26/what-is-zcash-the-privacy-coin-explained/](https://www.coindesk.com/layer2/2022/01/26/what-is-zcash-the-privacy-coin-explained/#:~:text=A%20trusted%20setup%20creates%20a,without%20revealing%20what%20it%20was)).

## Notation

Bold letters are vectors

$$
\langle\mathbf{a},\mathbf{b}\rangle
$$

Is an inner product between vectors **a** and **b**.

$$
[A]_1
$$

Is a G1 point

$$
[B]_2
$$

Is a G2 point

$$
I_{12}
$$

Is the identity in G12

Carry out the following computations in Python:

## Trusted Setup

Generate the powers of tau for A and B:

 

$$
[\tau^dG_1, \tau^{d-1}G_1,...,\tau G_1, G_1]
$$

$$
[\tau^dG_2, \tau^{d-1}G_2,...,\tau G_2, G_2]
$$

$$
[\dots,\tau^2t(\tau)G_1,\tau t(\tau)G_1,t(\tau)G_1]
$$

$$
\Psi_i=(w_i(\tau)+\alpha v_i(\tau)+\beta u_i(\tau))G_1 \space\space\space\text{for i = 1 to m}
$$

The trusted setup publishes

$$
\begin{align*}[\alpha]_1&=\alpha G_1 \\ [\beta]_2&=\beta G2 \\  \mathbf{\Psi}&=(w_i(\tau)+\alpha v_i(\tau)+\beta u_i(\tau))G_1\vert_{i=1}^m\end{align*}
$$

## Prover

The prover computes their witness vector **a** and computes:

$$
[A]_1 = [\alpha]_1 +\sum_{i=1}^m a_iu_i(\tau)
$$

$$
[B]_2 = [\beta]_2+\sum_{i=1}^m a_i v_i(\tau)
$$

$$
[C]_1 = \sum_{i=0}^m a_i\Psi_i+\langle \mathbf{h}, \mathbf{\eta} \rangle
$$

## Verifier

The verifier computes:

$$
\text{I}_{12}\stackrel{?}{=} \text{neg}([A]_1)\cdot [B]_2+[\alpha]_1\cdot [\beta]_2+[C]_1G_2
$$

The final step can be computed as

```solidity
from py_ecc.fields import optimized_bn128_FQ12 as FQ12
from py_ecc.bn128 import eq, neg, pairing, final_exponentiate

eq(FQ12.one(), final_exponentiate(pairing(B, neg(A)) * pairing(Beta, Alpha) * pairing(G2, C)))
```

If you get very stuck, feel free to refer to past implementations by students (which have been published on GitHub).

In [1]:
import numpy as np
from py_ecc.bn128 import G1, G2, pairing, multiply, add, eq
from py_ecc.bn128.bn128_curve import b, b2
import numpy as np
import galois
import functools
import galois 
import numpy as np
from py_ecc.bn128 import curve_order
from typing import Callable
import sympy as sp
from functools import partial
# Use the curve_order from py_ecc.bn128
GF = galois.GF(curve_order, primitive_element=5, verify=False)


In [2]:
def mat_to_finite_field(mat : np.array) -> np.array:
    return (mat + curve_order) % curve_order

In [3]:
# Define the matrices
A = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

B = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

C = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick values for x and y
x = 100
y = 100

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3
# the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])

F_A = GF(mat_to_finite_field(A))
F_B = GF(mat_to_finite_field(B))
F_C = GF(mat_to_finite_field(C))
F_W = GF(mat_to_finite_field(w))
n_rows = A.shape[0]


t = galois.Poly([1], field=GF)
for i in range(1, n_rows + 1):
    t *= galois.Poly([1, curve_order - i], field=GF)

def t_fn(tao : int, t : galois.Poly):
    return int(t(tao))


print(A.shape)
print(B.shape)
print(C.shape)
print(w.shape)
# do the trusted setup

(3, 6)
(3, 6)
(3, 6)
(6,)


In [14]:
#some helper functions

def poly_to_np(poly : galois.Poly, n_rows : int)-> np.array:
    elms = [int(coeff) for coeff in poly.coeffs]
    if len(elms) < n_rows:
        elms = elms + [0] * (n_rows - len(elms)) # pad with zeros if not enough elements
    return np.array(elms)

def pol_interp(mat : np.array) -> np.array:
    n_rows = mat.shape[0]
    def interpolate_column(col):
        xs = GF(np.arange(1, n_rows + 1))
        return galois.lagrange_poly(xs, col)
    
    interpolated = np.apply_along_axis(interpolate_column, 0, mat)
    return interpolated


def poly_to_np(poly : galois.Poly, n_rows : int)-> np.array:
    elms = [int(coeff) for coeff in poly.coeffs]
    if len(elms) < n_rows:
        elms = elms + [0] * (n_rows - len(elms)) # pad with zeros if not enough elements
    return np.array(elms)


In [5]:
# do the trusted setup
def trusted_setup(
    tao : int,
    alpha : int,
    beta : int,
    t: Callable[[int], int],
    n_rows : int,
    n_cols : int,
    u_polys : np.array,
    v_polys : np.array,
    w_polys : np.array,
) -> tuple[
    list[G1],
    list[G2],
    list[G1],
    list[G1],
    G1,
    G2,
]:
    '''
    $$
    \begin{align*}[\alpha]_1&=\alpha G_1 \\ [\beta]_2&=\beta G2 \\  \mathbf{\Psi}&=(w_i(\tau)+\alpha v_i(\tau)+\beta u_i(\tau))G_1\vert_{i=1}^m\end{align*}
    $$
    '''
    
    # [tao^dG1, tao^(d-1)G1, ..., taoG1, G1]
    srs1_int = [tao**i for i in range(n_rows-1, -1, -1)]
    srs1 = [multiply(G1, integer) for integer in srs1_int]
    # [tao^dG2, tao^(d-1)G2, ..., taoG2, G2]
    srs2_int = [tao**i for i in range(n_rows-1, -1, -1)]
    srs2 = [multiply(G2, integer) for integer in srs2_int]
    #[\dots,\tau^2t(\tau)G_1,\tau t(\tau)G_1,t(\tau)G_1]
    srs3_int = [(tao**i)*t(tao) for i in range(n_rows-1, -1, -1)]
    srs3 = [multiply(G1, integer) for integer in srs3_int]
    #\begin{align*}[\alpha]_1&=\alpha G_1 \\ [\beta]_2&=\beta G2 \\  \mathbf{\Psi}&=(w_i(\tau)+\alpha v_i(\tau)+\beta u_i(\tau))G_1\vert_{i=1}^m\end{align*}
    thetas = [w_polys[i](tao) + alpha * v_polys[i](tao) + beta * u_polys[i](tao) for i in range(n_cols)]
    psi = [multiply(G1, int(thetas[i])) for i in range(n_cols)]
    print([int(theta) for theta in thetas])
    A_alpha = multiply(G1, alpha)
    B_beta = multiply(G2, beta)
    
    return srs1, srs2, srs3, psi, A_alpha, B_beta

u_polys = pol_interp(GF(mat_to_finite_field(A)))
v_polys = pol_interp(GF(mat_to_finite_field(B)))
w_polys = pol_interp(GF(mat_to_finite_field(C)))


srs1, srs2, srs3, psi, A_alpha, B_beta = trusted_setup(
    tao=2, 
    alpha=1, 
    beta=1, 
    t=partial(t_fn, t=t), 
    n_rows=A.shape[0],
    n_cols=A.shape[1],
    u_polys=u_polys,
    v_polys=v_polys,
    w_polys=w_polys,
)


[0, 0, 0, 1, 1, 1]


' srs1_int, srs2_int, srs3_int, psi_int, A_alpha, B_beta = trusted_setup_integer(\n    tao=2, \n    alpha=1, \n    beta=1, \n    t=partial(t_fn, t=t), \n    n_rows=A.shape[0],\n    n_cols=A.shape[1],\n    u_polys=u_polys,\n    v_polys=v_polys,\n    w_polys=w_polys,\n) '

In [6]:
def eval_poly(coeffs: list[int], point: list):
    val = None    
    for i, coeff in enumerate(coeffs):
        if val is None:
            val = multiply(point[i], int(coeff))
        else:
            val = add(val, multiply(point[i], int(coeff)))
    
    return val

def poly_to_np(poly : galois.Poly, n_rows : int)-> np.array:
    elms = [int(coeff) for coeff in poly.coeffs]
    if len(elms) < n_rows:
        elms = elms + [0] * (n_rows - len(elms)) # pad with zeros if not enough elements
    return np.array(elms)

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return functools.reduce(sum_, map(mul_, polys, witness))


## Prover

The prover computes their witness vector **a** and computes:

$$
[A]_1 = [\alpha]_1 +\sum_{i=1}^m a_iu_i(\tau)
$$

$$
[B]_2 = [\beta]_2+\sum_{i=1}^m a_i v_i(\tau)
$$

$$
[C]_1 = \sum_{i=0}^m a_i\Psi_i+\langle \mathbf{h}, \mathbf{\eta} \rangle
$$

In [7]:
def compute_H(
    u_polys : np.array,
    v_polys : np.array,
    w_polys : np.array,
    witness : np.array,
    n_rows : int,
) -> np.array:
    term1 = inner_product_polynomials_with_witness(u_polys, witness)
    term2 = inner_product_polynomials_with_witness(v_polys, witness)
    term3 = inner_product_polynomials_with_witness(w_polys, witness)
    t = galois.Poly([1], field=GF)
    for i in range(1, n_rows + 1):
        t *= galois.Poly([1, curve_order - i], field=GF)

    h = (term1 * term2 - term3) // t
    return h

In [8]:


def prover(
    witness : np.array,
    srs1 : list[G1],
    srs2 : list[G2],
    psi : list[G1],
    A_alpha : G1,
    u_polys : np.array,
    v_polys : np.array,
    w_polys : np.array,
    B_beta : G2,
    n_rows : int,
) -> tuple[G1, G2, G1]:
    
    u_scaled = inner_product_polynomials_with_witness(u_polys, witness)
    v_scaled = inner_product_polynomials_with_witness(v_polys, witness)
    

    
    H = compute_H(u_polys, v_polys, w_polys, witness, n_rows)

    poly_h = poly_to_np(H, n_rows)
    poly_A = poly_to_np(u_scaled, n_rows)
    poly_B = poly_to_np(v_scaled, n_rows)
    
    A = add(eval_poly(poly_A, srs1), A_alpha)
    B = add(eval_poly(poly_B, srs2), B_beta)
    
    accum = eval_poly(poly_h, srs3)
    C = add(eval_poly(witness, psi), accum) 

    return A, B, C
    
PROVER_A, PROVER_B, PROVER_C = prover(
    witness=F_W,
    srs1=srs1,
    srs2=srs2,
    psi=psi,
    u_polys=u_polys,
    v_polys=v_polys,
    w_polys=w_polys,
    A_alpha=A_alpha,
    B_beta=B_beta,
    n_rows=A.shape[0],
)

In [13]:


def verifier(
    A : G1,
    B : G2,
    C : G1,
    Beta : G2,
    Alpha : G1,
) -> bool:
    #from py_ecc.fields import optimized_bn128_FQ12 as FQ12
    
    from py_ecc.bn128 import eq, neg, pairing, final_exponentiate, FQ12
    return eq(FQ12.one(), final_exponentiate(pairing(B, neg(A)) * pairing(Beta, Alpha) * pairing(G2, C)))

In [12]:
res =verifier(
    A=PROVER_A,
    B=PROVER_B,
    C=PROVER_C,
    Beta=B_beta,
    Alpha=A_alpha,
)
print("res", res)


res True


' \nverifier_integer(\n    A=1,\n    B=1,\n    C=1,\n    Beta=1,\n    Alpha=1,\n) '